In [7]:
import re, string, collections
import pandas as pd
import matplotlib.pyplot as plt
import nltk

#from wordcloud import WordCloud
#from nltk import SnowballStemmer
#from nltk.corpus import stopwords
#nltk.download('stopwords')
import pickle
import numpy as np
import pprint as p

In [82]:
def cleanData (file, symbols=''):
    tweets = pd.read_csv(file, header=0, names=['ID', 'tweet'], sep='\t')
    #tweets = pd.read_csv(r'train.tsv', sep='\t')
    data = tweets.get('tweet').tolist()
    clean_data = list()
    for tweet in data:
        url = re.sub(r'http\S+','', tweet)
        simbolos = re.sub(symbols, '', url)
        usuarios = re.sub(r'@\w+', '_user_', simbolos)
        hashtags = re.sub(r'#\w+', '_hashtag_', usuarios)
        clean_data.append(hashtags.lower())
    return data, clean_data

def markData (data):
    frases_pattern = re.compile(
        "(\w+)( )?"
        "(\.|,|;|\?|!)+"
        "( )?(\w+)"
    )
    mark_data = list()
    for tweet in data:
        tweet = re.sub(frases_pattern, r'\1 _EOS_  _BOS_ \5', tweet, flags=0) #Lo deja para separa ideas
        tweet = ' _BOS_ ' + tweet + ' _EOS_ '
        tweet = re.sub('(\.|,|;|\?|!)+', r'', tweet, flags=0)
        mark_data.append(tweet)
    return mark_data

def createTokens(mark_data):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(mark_data)
    frequencies = collections.Counter(tokens)
    unique_tokens = [item[0] for item in filter(lambda x: x[1] == 1, frequencies.items())]
    return tokens, unique_tokens


def createVocabulary (tokens, unique_tokens):
    vocabulario = list(set(tokens)-set(unique_tokens))
    size = len(vocabulario)
    dic_index = dict(zip(range(size),vocabulario))  #Índice -> Palabra
    dic_index[size] = '_OOV_' 
    dic_words = dict(zip(vocabulario, range(size))) #Palabra -> Índice
    dic_words['_OOV_'] = size
    return vocabulario, dic_index, dic_words


    
def genBigrams(dic_words,tokens,vocabulario):
    separador = (dic_words['_EOS_'],dic_words['_BOS_'])
    ovv = dic_words['_OOV_']
    tokens_idx = [dic_words[token] if token in vocabulario else ovv for token in tokens]
    #print(tokens_idx)
    bigrams = list()
    for i in range(len(tokens_idx) - 1):
        if tokens_idx[i] != ovv:
            bigrams.append((tokens_idx[i], tokens_idx[i+1]))
    return bigrams
    
    

def dropUniqueTokens(tokens,dic_words):
    frequencies = collections.Counter(tokens)
    unique_tokens = [item[0] for item in filter(lambda x: x[1] == 1, frequencies.items())]
    delete = [(dic_words['_EOS_'],dic_words['_BOS_'])]
    for unique in unique_tokens:
        index = tokens.index(unique)
        delete.append((dic_words[tokens[index]],dic_words[tokens[index+1]]))
        delete.append((dic_words[tokens[index-1]],dic_words[tokens[index]]))
    return delete, unique_tokens

#Sub dividir la probabilidad de OVV en cada palabra.

In [91]:
puntuacion = r'[\}\{¡"$%&\'()¿:=\+[\]*-]'

data, clean_data = cleanData(r'train_MX.tsv', puntuacion)
mark_data = markData(clean_data)
tokens, unique_tokens = createTokens(' '.join(mark_data))
vocabulario, dic_index, dic_words = createVocabulary(tokens, unique_tokens)
bigrams = genBigrams(dic_words,tokens,vocabulario)

print('Palabras únicas:',len(unique_tokens),'Número de tokens',len(tokens),'Tamaño no únicas',len(vocabulario))
print('Bigramas para entrenamiento: ',len(bigrams))
print(dic_words['_OOV_'])
len([x for x in bigrams if x[1] == dic_words['_OOV_']]) #Tamaño de token -> OOV

Palabras únicas: 3416 Número de tokens 31546 Tamaño no únicas 1728
Bigramas para entrenamiento:  28129
1728


3094

In [92]:
with open('Corpus.pk', 'wb') as corpus_file:
    pickle.dump(dic_words, corpus_file)
    pickle.dump(dic_index, corpus_file)
    pickle.dump(vocabulario, corpus_file)
    pickle.dump(bigrams, corpus_file)
    pickle.dump(tokens, corpus_file)
    pickle.dump(unique_tokens, corpus_file)

In [6]:

for a,b in bigrams[:25]:
    print(a,b)
    print(dic_index[a],dic_index[b],"\n")
phrase

0 2
_B_ sadness 

2 3
sadness _E_ 

0 1
_B_ joy 

1 3
joy _E_ 

0 1
_B_ joy 

1 3
joy _E_ 

0 5
_B_ others 

5 3
others _E_ 

0 2
_B_ sadness 

2 3
sadness _E_ 

0 1
_B_ joy 

1 3
joy _E_ 

0 1
_B_ joy 

1 3
joy _E_ 

0 5
_B_ others 

5 3
others _E_ 

0 2
_B_ sadness 

2 3
sadness _E_ 

0 5
_B_ others 

5 3
others _E_ 

0 5
_B_ others 

5 3
others _E_ 

0 5
_B_ others 

5 3
others _E_ 

0 2
_B_ sadness 



(3, 0)

In [38]:
print("Texto Original:\n",data[66])
print("Texto Limpio:\n",clean_data[66])
print("Texto Marcado:\n",mark_data[66],"\n")

print("Texto Original:\n",data[71])
print("Texto Limpio:\n",clean_data[71])
print("Texto Marcado:\n",mark_data[71],"\n")

print("Texto Original:\n",data[114])
print("Texto Limpio:\n",clean_data[114])
print("Texto Marcado:\n",mark_data[114],"\n")

Texto Original:
 #BullyingAJos "Mi maestra me dio un beso en la salida, por que hice los palitos derechitos..." Pa!rodia! @JosDice
Texto Limpio:
 _hashtag_ mi maestra me dio un beso en la salida, por que hice los palitos derechitos... pa!rodia! _user_
Texto Marcado:
  _BOS_ _hashtag_ mi maestra me dio un beso en la salida _EOS_  _BOS_ por que hice los palitos derechitos _EOS_  _BOS_ parodia _EOS_  _BOS_ _user_ _EOS_  

Texto Original:
 ¡Ay como sufren!!! Desde tempranito empezaron a cortarse las venas... Mejor vayan a las posadas, aunque sea pescan a la señora que reza
Texto Limpio:
 ay como sufren!!! desde tempranito empezaron a cortarse las venas... mejor vayan a las posadas, aunque sea pescan a la señora que reza
Texto Marcado:
  _BOS_ ay como sufren _EOS_  _BOS_ desde tempranito empezaron a cortarse las venas _EOS_  _BOS_ mejor vayan a las posadas _EOS_  _BOS_ aunque sea pescan a la señora que reza _EOS_  

Texto Original:
 Si definieran su primer día del 2017 con una palabra cuál 